In [1]:
from torch.utils.data import DataLoader
from torch import optim
import numpy as np
import time
import csv
import warnings

from lib.aperture_dataset import ApertureDataset
from lib.lenet import LeNet
from lib.fit import fit

log_fname = 'results_%s.csv' % time.strftime('%Y%m%d-%H%M%S')

k = 4
fname_train = 'training_data/training_data.h5'
fname_validate = 'training_data/validation_data.h5'
save_path = 'save_here'


num_samples = 10 ** 5
dat_train = ApertureDataset(fname_train, num_samples, k)

num_samples = 10 ** 4
dat_train2 = ApertureDataset(fname_train, num_samples, k)

num_samples = 10 ** 4
dat_validate = ApertureDataset(fname_validate, num_samples, k)


/Users/zhanwenchen/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# TODO: Implement Dropout

using_cuda = False
if not using_cuda: warnings.warn("Not using CUDA")

# optimization parameters
try_learning_rates = [1.0, 0.75, 0.5, 0.25, 0.1, 0.05, 0.01, 0.005, 0.001]
optimization_algo = "Momentum"
if optimization_algo == "Momentum": momentum = 0.9
try_batch_sizes = [32, 64, 128]
# NOTE we are using early stopping where we stop training once validation cost stops decreasing.


# Model parameters
input_size = 65
output_size = 130

# conv1
try_conv1_kernel_sizess = list(range(2, 10))
try_conv1_num_kernels = list(range(2, 33))
try_conv1_strides = [1, 2]

# pool1
try_pool1_sizes = [2, 3]
# try_pool1_sizes = [2] # TODO: support pool_size = 3 in lenet.

# conv2
# TODO in loop: conv2_kernel_size should <= conv1_kernel_size
# TODO in loop: conv2_num_kernels should >= conv1_num_kernels
try_conv2_kernel_sizess = list(range(2, 10))
try_conv2_num_kernels = list(range(2, 33))
try_conv2_strides = [1, 2]

# pool2
try_pool2_sizes = [2, 3]

# fcs
try_fc_hidden_sizes = [8, 16, 32, 64, 128, 256, 512, 1024]
try_fc_num_hidden_layers = [1, 2, 3]
                    

# Random Hyperparameter Search
def choose(array): return int(np.random.choice(array))

num_trys = 100
run_counter = 0

print()
print("| run | testing loss | diff     | learning_rate | batch_size | conv1_kernel_size | conv1_num_kernels | conv1_stride | pool1_kernels | conv2_kernel_size | conv2_kernels | conv2_strides | pool2_kernels | fc1_size |")
print("| --- | ------------ | -------- | ------------- | ---------- | ----------------- | ----------------- | ------------ | ------------- | ----------------- | ------------- | ------------- | ------------- | -------- |")

while run_counter < num_trys:
    # choose random hyperparameters: optimization
    batch_size = int(np.random.choice(try_batch_sizes))
    learning_rate = int(np.random.choice(try_learning_rates))
    
    # choose random hyperparameters: model
    conv1_kernel_size = choose(try_conv1_kernel_sizess)
    conv1_num_kernels = choose(try_conv1_num_kernels)
    conv1_stride = choose(try_conv1_strides)
    
    # enforce relative shape and divisibility
    conv1_output_size = (conv1_num_kernels, (input_size - conv1_kernel_size) / conv1_stride + 1)
    while isinstance(conv1_output_size[1], float) and not conv1_output_size[1].is_integer():
        # conv1_output_size must be an integer
        # random search is not the most efficient approach but I'm too lazy to filter right now.
        conv1_stride = choose(try_conv1_strides)
        conv1_output_size = (conv1_num_kernels, (input_size - conv1_kernel_size) / conv1_stride + 1)
#         print('loop: conv1_output_size =', conv1_output_size)
        
    
    pool1_kernel_size = choose(try_pool1_sizes)
    pool1_stride = 2
#     print("loop: (conv1_output_size[1] - pool1_kernel_size) =", (conv1_output_size[1] - pool1_kernel_size))
    pool1_output_size = (conv1_num_kernels, (conv1_output_size[1] - pool1_kernel_size) / pool1_stride + 1)
    while isinstance(pool1_output_size[1], float) and not pool1_output_size[1].is_integer():
        # conv1_output_size must be an integer
        pool1_kernel_size = choose(try_pool1_sizes)
        pool1_output_size = (conv1_num_kernels, (conv1_output_size[1] - pool1_kernel_size) / pool1_stride + 1)
#         print("loop: (conv1_output_size[1] - pool1_kernel_size) =", (conv1_output_size[1] - pool1_kernel_size))
#         print('loop: pool1_output_size =', pool1_output_size)
    
    conv2_kernel_size = choose(try_conv2_kernel_sizess)
    conv2_num_kernels = choose(try_conv2_num_kernels)
    conv2_stride = choose(try_conv2_strides)
    
    conv2_output_size = (conv2_num_kernels, (pool1_output_size[1] - conv2_kernel_size) / conv2_stride + 1)
    while isinstance(conv2_output_size[1], float) and not conv2_output_size[1].is_integer():
        # conv2_output_size must be an integer
        conv2_stride = choose(try_conv2_strides)
        conv2_output_size = (conv2_num_kernels, (pool1_output_size[1] - conv2_kernel_size) / conv2_stride + 1)
#         print('loop: conv2_output_size =', conv2_output_size)
    
    pool2_kernel_size = choose(try_pool2_sizes)
    pool2_stride = 2
    pool2_output_size = (conv2_num_kernels, (conv2_output_size[1] - pool2_kernel_size) / pool2_stride + 1)
    
    while isinstance(pool2_output_size[1], float) and not pool2_output_size[1].is_integer():
        # conv1_output_size must be an integer
        pool2_kernel_size = choose(try_pool2_sizes)
        pool2_output_size = (conv2_num_kernels, (conv2_output_size[1] - pool2_kernel_size) / pool2_stride + 1)
#         print('loop: pool2_output_size =', pool2_output_size)
    
    fcs_hidden_size = choose(try_fc_hidden_sizes)
    fcs_num_hidden_layers = choose(try_fc_num_hidden_layers)
    
    # load data because of difference batch_sizes
    train_loader = DataLoader(dat_train, batch_size=batch_size, shuffle=True, num_workers=1)
    train_loader2 = DataLoader(dat_train2, batch_size=batch_size, shuffle=False, num_workers=1)
    validate_loader = DataLoader(dat_validate, batch_size=batch_size, shuffle=False, num_workers=1)
    
    # build the model and train
    model = LeNet(input_size, output_size, fcs_hidden_size, fcs_num_hidden_layers,
                  batch_size,
                  pool1_kernel_size,
                  conv1_kernel_size, conv1_num_kernels, conv1_stride,
                  pool2_kernel_size,
                  conv2_kernel_size, conv2_num_kernels, conv2_stride)
    
    if using_cuda: model.cuda()
    optimizer = optim.SGD(model.parameters(), learning_rate, momentum)
    cost, diff = fit(model, train_loader, train_loader2, validate_loader, optimizer, save_path, cuda=using_cuda)

    print("| %02d  | %.10f | %0.6f | %.10f  | %03d        | %02d                | %02d                 | %d            | %d             | %d                | %d             | %d             | %d           | %04d     |" % \
          (run_counter,cost, diff,learning_rate,batch_size,conv1_kernel_size,  conv1_num_kernels,      conv1_stride,   pool1_kernels,    conv2_kernel_size,  conv2_kernels,   conv2_strides, pool2_kernels,    fc1_size))
    run_counter += 1

# for batch_size in try_batch_sizes:

#     train_loader = DataLoader(dat_train, batch_size=batch_size, shuffle=True, num_workers=1)
#     train_loader2 = DataLoader(dat_train2, batch_size=batch_size, shuffle=False, num_workers=1)
#     validate_loader = DataLoader(dat_validate, batch_size=batch_size, shuffle=False, num_workers=1)        

#     for kernel_size in try_kernel_sizes:
#         # TODO: in lenet.py, change fc layer input_size to allow any num_kernels
#         for num_kernels in try_num_kernels:

#             for num_hidden_layers in try_num_hidden_layers:

#                 for hidden_size in try_hidden_sizes:

#                     model = LeNet(input_dim, output_dim, hidden_size, num_hidden_layers, batch_size)
#                     optimizer = optim.SGD(model.parameters(), lr, momentum)
#                     if using_cuda: model.cuda()
#                     cost, diff = fit(model, train_loader, train_loader2, validate_loader, optimizer, save_path, cuda=using_cuda)

#                     row = '%03d, %0.5f, %2.3f%%, %02d, %02d, %d, %03d, %03d' % (run_index, cost, diff, kernel_size, num_kernels, num_hidden_layers, hidden_size, batch_size)

#                     print("| %03d | %0.5f    | %2.3f%% | %02d          | %02d          | %d                 | %03d         | %03d        |" % (run_index, cost, diff, kernel_size, num_kernels, num_hidden_layers, hidden_size, batch_size))
#                     writer.writerow(row)

#                     run_index += 1
    

print("\nIt's all over.")


| run | testing loss | diff     | learning_rate | batch_size | conv1_kernel_size | conv1_num_kernels | conv1_stride | pool1_kernels | conv2_kernel_size | conv2_kernels | conv2_strides | pool2_kernels | fc1_size |
| --- | ------------ | -------- | ------------- | ---------- | ----------------- | ----------------- | ------------ | ------------- | ----------------- | ------------- | ------------- | ------------- | -------- |
lenet: conv1_output_size = (18, 58.0)
lenet: pool1_output_size = (58 - 2 ) / 2 + 1 = 29.0
lenet: pool1_output_size = (18, 29)
lenet: conv2_output_size = (20, 26.0)
lenet: pool2_kernel_size = (20, 13.0)
lenet: fcs_input_size = 260


/Users/zhanwenchen/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Not using CUDA
  after removing the cwd from sys.path.


NameError: name 'pool1_kernels' is not defined

In [ ]:
# num_trys = 1000

# run_counter = 1

# print()
# print("| run | testing loss | accuracy | learning_rate | epochs | batch_size | dropout | conv1_filters | conv1_kernels | conv1_strides | pool1_kernels | conv2_filters | conv2_kernels | conv2_strides | pool2_kernels | fc1_size |")
# print("| --- | ------------ | -------- | ------------- | ------ | ---------- | ------- | ------------- | ------------- | ------------- | ------------- | ------------- | ------------- | ------------- | ------------- | -------- |")

# while run_counter <= num_trys:
#     learning_rate = np.random.uniform(0.001, 0.5)
#     batch_size = np.random.choice([16, 32, 64, 128, 256])
#     num_epochs = np.random.choice([5, 10, 15, 20])
# #     num_epochs = 1
#     dropout = np.random.uniform(0.05, 0.5)
#     conv1_filters = np.random.randint(1, 100)
#     conv1_kernels = int(np.random.choice([3, 5, 7]))
#     conv1_strides = int(np.random.choice([1, 2]))
    
#     pool1_kernels = int(np.random.choice([2, 3]))
# #     print()
# #     print("pool1_kernels", pool1_kernels)
    
#     conv2_filters = np.random.randint(conv1_filters, 100)
#     conv2_kernels = 3 if conv1_kernels == 3 else int(np.random.choice([3, 5]))
#     conv2_strides = int(np.random.choice([1, 2]))
    
#     pool2_kernels = int(np.random.choice([2, 3]))
# #     print("pool2_kernels", pool2_kernels)
    
#     fc1_size = int(np.random.choice([8, 16, 32, 64, 128]))
    
#     train_loader = DataLoader(dat_train, batch_size=batch_size, shuffle=True, num_workers=1)
#     train_loader2 = DataLoader(dat_train2, batch_size=batch_size, shuffle=False, num_workers=1)
#     validate_loader = DataLoader(dat_validate, batch_size=batch_size, shuffle=False, num_workers=1) 
    
    
#     try:
# #         print("success")
#         example_model_fn = get_model_fn(10, dropout, conv1_filters, conv1_kernels, conv1_strides, conv2_filters, conv2_kernels, conv2_strides, pool1_strides, pool1_kernels, pool2_strides, pool2_kernels, fc1_size, learning_rate)
#         model = tf.estimator.Estimator(example_model_fn)



#         # Define the input function for training
#         input_fn = tf.estimator.inputs.numpy_input_fn(
#             x={'images': train_images}, y=train_labels,
#             batch_size=batch_size, num_epochs=num_epochs, shuffle=True)
#         # Train the Model
#         model.train(input_fn, steps=None)

#         # Evaluate the Model
#         # Define the input function for evaluating
#         input_fn = tf.estimator.inputs.numpy_input_fn(
#             x={'images': test_images}, y=test_labels,
#             batch_size=batch_size, shuffle=False)
#         # Use the Estimator 'evaluate' method
#         e = model.evaluate(input_fn)
#         accuracy = e['accuracy']
#         loss = e['loss']

#         print("| %02d  | %.10f | %0.6f | %.10f  | %03d    | %03d        | %.3f   | %02d            | %d             | %d             | %d             | %d            | %d             | %d             | %d           | %04d     |" % \
#           (run_counter, loss,  accuracy,learning_rate,num_epochs,batch_size,dropout,conv1_filters,  conv1_kernels,  conv1_strides,   pool1_kernels,    conv2_filters,  conv2_kernels,   conv2_strides, pool2_kernels,    fc1_size))

#         run_counter += 1
#     except:
# #         print("failure")
#         continue